In [50]:

import spacy
nlp = spacy.load('ja_ginza')
doc = nlp('依存構造文の解析をGinzaで行います。')
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.i)
    print('EOS')

0 依存 依存 NOUN 名詞-普通名詞-サ変可能 compound 2
1 構造 構造 NOUN 名詞-普通名詞-一般 compound 2
2 文 文 NOUN 名詞-普通名詞-一般 nmod 4
3 の の ADP 助詞-格助詞 case 2
4 解析 解析 NOUN 名詞-普通名詞-サ変可能 obj 8
5 を を ADP 助詞-格助詞 case 4
6 Ginza ginza NOUN 名詞-普通名詞-一般 nmod 8
7 で で ADP 助詞-格助詞 case 6
8 行い 行う VERB 動詞-一般 ROOT 8
9 ます ます AUX 助動詞 aux 8
10 。 。 PUNCT 補助記号-句点 punct 8
EOS


In [51]:
import json
from elasticsearch import Elasticsearch
import os
es = Elasticsearch("localhost:9200")

In [56]:

with open('qa_train2.json', 'r',encoding="utf-8") as f:
    qa_train = json.load(f)



for qa in qa_train['data']:
    es.index(index="qa_train2", 
             doc_type="test", 
             body={
                 "question":qa['paragraphs'][0]['qas'][0]['question'], 
                 "context":qa['paragraphs'][0]['context'],
                 "answer":qa['paragraphs'][0]['qas'][0]['answers'][0]['text'],
                 "id":qa['title']
             })

In [55]:
es.indices.delete(index="qa_train1")

{'acknowledged': True}

In [57]:
with open('qa_dev2.json', 'r',encoding="utf-8") as f:
    qa_dev = json.load(f)
    
qa_dev = qa_dev['data']
qa_dev[0]['paragraphs'][0]['qas'][0]['question']

'バイアスとはなんですか'

In [58]:
es = Elasticsearch("localhost:9200")



true_count = 0
false_count = 0


search_top_5 = []  # { 'id':'0101', 
                    #    'q':'多階層ニューラルネットワークとは何か', 
                    #    'ruizido' : [
                    #       { 'top1_id': '0102, 'top1_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #       { 'top2_id': '0102, 'top2_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #          ...
                    #     ]

for idx in range(len(qa_dev)):
    
# sudachiを使うver    
#     body = {
#         "query" : {
#             "bool": {
#                 "must": [
#                     {
#                       "query_string": {
#                         "analyzer": "sudachi_analyzer",
#                         "query": qa_dev[idx]['paragraphs'][0]['qas'][0]['question']
#                       }
#                     }
#                 ]            
#             }
#         },
#         "highlight": {
#             "fields": {
#                 "itemCaption": {}
#             }
#         }
#     }

#  sudachiを使わないver
    body = {
        "query" : {
            "bool": {
                "must": [
                ]            
            }
        },
        "highlight": {
            "fields": {
                "itemCaption": {}
            }
        }
    }

    body['query']['bool']['must'].append(
        { 
            "bool": {
                "should": [
                    { "match": { "question": qa_dev[idx]['paragraphs'][0]['qas'][0]['question'] } }
                ]
            }
        }
    )

    print("------------devの質問---------------")
    print(qa_dev[idx]['title'], qa_dev[idx]['paragraphs'][0]['qas'][0]['question'])
    print("------------trainの検索結果------------")
    result = es.search(index='qa_train2', body=body, size=1000) # ここをかえる！！！！
    result_num = result['hits']['total']['value']
    get_qa_train = result['hits']['hits']
    print(get_qa_train[0]['_source']['id'],get_qa_train[0]['_source']['question'])
    print(get_qa_train[1]['_source']['id'],get_qa_train[1]['_source']['question'])
    print(get_qa_train[2]['_source']['id'],get_qa_train[2]['_source']['question'])
    
    
    # ruizido_top_5に格納する辞書型を作る
    mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_qa_train[0]['_source']['id'], 'top1_ans':  get_qa_train[0]['_source']['question'] },
                        { 'top2_id':  get_qa_train[1]['_source']['id'], 'top2_ans':  get_qa_train[1]['_source']['question'] },
                        { 'top3_id':  get_qa_train[2]['_source']['id'], 'top3_ans':  get_qa_train[2]['_source']['question'] },
                        { 'top4_id':  get_qa_train[3]['_source']['id'], 'top4_ans':  get_qa_train[3]['_source']['question'] },
                        { 'top5_id':  get_qa_train[4]['_source']['id'], 'top5_ans':  get_qa_train[4]['_source']['question'] }
          ]
          }
    search_top_5.append(mydict)
    
    if qa_dev[idx]['title'] == get_qa_train[0]['_source']['id']:
        true_count += 1
    else:
        false_count += 1


print("------結果-------")
print(true_count)
print(false_count)
print("精度：", true_count / (true_count + false_count))

------------devの質問---------------
0610 バイアスとはなんですか
------------trainの検索結果------------
0610 バイアスってなんですか
0610 バイアスと分散はどのような関係ですか
0305 バイアスって何ですか
------------devの質問---------------
0306 候補削除アルゴリズムとはなんですか
------------trainの検索結果------------
0306 候補削除アルゴリズムってなんですか
0204 次元削減とはなんですか
1116 EMアルゴリズムとはなんですか
------------devの質問---------------
1502 強化学習とは何ですか
------------trainの検索結果------------
1502 強化学習とはなんですか
1501 強化学習はなぜ中間的学習なのですか
1502 強化学習は例えば何に使われていますか
------------devの質問---------------
0605 決定係数の定義は何ですか
------------trainの検索結果------------
0605 決定係数ってなんですか
0701 マージンの定義はなんですか
0901 深層学習の定義はなにか
------------devの質問---------------
0916 時系列信号や自然言語などの系列パターンを扱うことができるニューラルネットワークは何ですか
------------trainの検索結果------------
1301 入力の系列長と出力の系列長の間に明確な対応関係がないとはどういうことですか
1301 系列データの入力の系列長と出力の系列長が等しい問題の解決法はどのようなものですか
1301 系列データの入力の系列長にかかわらず，出力の系列長が1である問題とはつまりどのような問題ですか
------------devの質問---------------
1502 強化学習はどのような学習法ですか
------------trainの検索結果------------
1503 強化学習で、報酬が決定的な場合の学習はどのようなものですか
1501 強化学習はなぜ中間的学習なのですか
1502 

0917 リカレントニューラルネットワークはどのようにして勾配消失問題を解決しているか
0504 生成モデルアプローチが有効なのはどのようなときですか
1001 なぜ仮定として，識別器の誤り率はすべて等しく，その誤りは独立であるとするのですか
------------devの質問---------------
0908 自己写像の学習において使われる入力と出力の距離は何ですか
------------trainの検索結果------------
0908 深層学習における初期パラメータの調整で必要な，入力の情報をなるべく失わない，より少ないノードへの写像を学習でよく使われる手段はなに
0908 入力が0または1の2値であれば，出力層の活性化関数として使われるのはなんですか
1309 入力の系列長に関わらず出力の系列長が1である問題は何が難しいのですか
------------devの質問---------------
0907 事前学習法の考え方は何ですか
------------trainの検索結果------------
1412 ラベル伝搬法の考え方は何ですか
0907 事前学習法とは何ですか
0907 事前学習とは何ですか
------------devの質問---------------
0611 回帰木ってなんですか
------------trainの検索結果------------
0611 回帰木って何ですか
0606 Ridge回帰ってなんですか
0607 Lasso回帰ってなんですか
------------devの質問---------------
1209 リフト値とは何ですか
------------trainの検索結果------------
1209 リフト値ってなんですか
1209 リフト値の値からどのようなことがわかりますか
1111 外れ値とは何ですか
------------devの質問---------------
1209 確信度の値からどのようなことがわかりますか
------------trainの検索結果------------
1209 リフト値の値からどのようなことがわかりますか
0114 クラスタリングはどのようなことに使われますか
1209 確信度とはなんですか
------------devの質問-----

In [59]:
search_top_5

[{'id': '0610',
  'q': 'バイアスとはなんですか',
  'search': [{'top1_id': '0610', 'top1_ans': 'バイアスってなんですか'},
   {'top2_id': '0610', 'top2_ans': 'バイアスと分散はどのような関係ですか'},
   {'top3_id': '0305', 'top3_ans': 'バイアスって何ですか'},
   {'top4_id': '0311', 'top4_ans': 'エントロピーとはなんですか'},
   {'top5_id': '1010', 'top5_ans': 'ブースティングとはなんですか'}]},
 {'id': '0306',
  'q': '候補削除アルゴリズムとはなんですか',
  'search': [{'top1_id': '0306', 'top1_ans': '候補削除アルゴリズムってなんですか'},
   {'top2_id': '0204', 'top2_ans': '次元削減とはなんですか'},
   {'top3_id': '1116', 'top3_ans': 'EMアルゴリズムとはなんですか'},
   {'top4_id': '0305', 'top4_ans': 'FIND-Sアルゴリズムとはなんですか'},
   {'top5_id': '1214', 'top5_ans': 'FP-Growthアルゴリズムとはなんですか'}]},
 {'id': '1502',
  'q': '強化学習とは何ですか',
  'search': [{'top1_id': '1502', 'top1_ans': '強化学習とはなんですか'},
   {'top2_id': '1501', 'top2_ans': '強化学習はなぜ中間的学習なのですか'},
   {'top3_id': '1502', 'top3_ans': '強化学習は例えば何に使われていますか'},
   {'top4_id': '1501', 'top4_ans': '強化学習が中間的学習という位置づけにある理由はなんですか'},
   {'top5_id': '1503', 'top5_ans': '強化学習で、報酬が決定的な場合の学習はどのようなもので

In [60]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data


pickle_dump(search_top_5, './search2_top_5.pickle')
print(pickle_load('./search2_top_5.pickle'))

[{'id': '0610', 'q': 'バイアスとはなんですか', 'search': [{'top1_id': '0610', 'top1_ans': 'バイアスってなんですか'}, {'top2_id': '0610', 'top2_ans': 'バイアスと分散はどのような関係ですか'}, {'top3_id': '0305', 'top3_ans': 'バイアスって何ですか'}, {'top4_id': '0311', 'top4_ans': 'エントロピーとはなんですか'}, {'top5_id': '1010', 'top5_ans': 'ブースティングとはなんですか'}]}, {'id': '0306', 'q': '候補削除アルゴリズムとはなんですか', 'search': [{'top1_id': '0306', 'top1_ans': '候補削除アルゴリズムってなんですか'}, {'top2_id': '0204', 'top2_ans': '次元削減とはなんですか'}, {'top3_id': '1116', 'top3_ans': 'EMアルゴリズムとはなんですか'}, {'top4_id': '0305', 'top4_ans': 'FIND-Sアルゴリズムとはなんですか'}, {'top5_id': '1214', 'top5_ans': 'FP-Growthアルゴリズムとはなんですか'}]}, {'id': '1502', 'q': '強化学習とは何ですか', 'search': [{'top1_id': '1502', 'top1_ans': '強化学習とはなんですか'}, {'top2_id': '1501', 'top2_ans': '強化学習はなぜ中間的学習なのですか'}, {'top3_id': '1502', 'top3_ans': '強化学習は例えば何に使われていますか'}, {'top4_id': '1501', 'top4_ans': '強化学習が中間的学習という位置づけにある理由はなんですか'}, {'top5_id': '1503', 'top5_ans': '強化学習で、報酬が決定的な場合の学習はどのようなものですか'}]}, {'id': '0605', 'q': '決定係数の定義は何ですか', 'searc

In [39]:
################################################################################################
#
#                                     以上フラットな検索
#
#                    以下ではチュータリングシステムにおける検索精度を測る
#
################################################################################################

In [61]:
es = Elasticsearch("localhost:9200")



true_count = 0
false_count = 0


tutoring_search_top_5 = []  # { 'id':'0101', 
                    #    'q':'多階層ニューラルネットワークとは何か', 
                    #    'ruizido' : [
                    #       { 'top1_id': '0102, 'top1_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #       { 'top2_id': '0102, 'top2_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #          ...
                    #     ]

for idx in range(len(qa_dev)):
    
# sudachiを使うver    
#     body = {
#         "query" : {
#             "bool": {
#                 "must": [
#                     {
#                       "query_string": {
#                         "analyzer": "sudachi_analyzer",
#                         "query": qa_dev[idx]['paragraphs'][0]['qas'][0]['question']
#                       }
#                     }
#                 ]            
#             }
#         },
#         "highlight": {
#             "fields": {
#                 "itemCaption": {}
#             }
#         }
#     }

#  sudachiを使わないver
    body = {
        "query" : {
            "bool": {
                "must": [
                ]            
            }
        },
        "highlight": {
            "fields": {
                "itemCaption": {}
            }
        }
    }

    body['query']['bool']['must'].append(
        { 
            "bool": {
                "should": [
                    { "match": { "question": qa_dev[idx]['paragraphs'][0]['qas'][0]['question'] } }
                ]
            }
        }
    )

#     print("------------devの質問---------------")
#     print(qa_dev[idx]['title'], qa_dev[idx]['paragraphs'][0]['qas'][0]['question'])
#     print("------------trainの検索結果------------")
    result = es.search(index='qa_train2', body=body, size=1000)
    result_num = result['hits']['total']['value']
    get_qa_train = result['hits']['hits']
    
    get_id_and_ans = []
    
    for one_get_qa_train in get_qa_train:
        
        if int(qa_dev[idx]['title']) >= int(one_get_qa_train['_source']['id']):
            
            get_id_and_ans.append([one_get_qa_train['_source']['id'], one_get_qa_train['_source']['question']])
            
        if len(get_id_and_ans) == 5:
            break
    
    
    if len(get_id_and_ans) == 1:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  get_id_and_ans[0][1] },
                        { 'top2_id':  None, 'top2_ans':  None },
                        { 'top3_id':  None, 'top3_ans':  None },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 2:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  get_id_and_ans[0][1] },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  get_id_and_ans[1][1] },
                        { 'top3_id':  None, 'top3_ans':  None },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 3:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  get_id_and_ans[0][1] },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  get_id_and_ans[1][1] },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans':  get_id_and_ans[2][1] },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 4:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  get_id_and_ans[0][1] },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  get_id_and_ans[1][1] },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans':  get_id_and_ans[2][1] },
                        { 'top4_id':  get_id_and_ans[3][0], 'top4_ans':  get_id_and_ans[3][1] },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
        
    if len(get_id_and_ans) == 5:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  get_id_and_ans[0][1] },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  get_id_and_ans[1][1] },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans':  get_id_and_ans[2][1] },
                        { 'top4_id':  get_id_and_ans[3][0], 'top4_ans':  get_id_and_ans[3][1] },
                        { 'top5_id':  get_id_and_ans[4][0], 'top5_ans':  get_id_and_ans[4][1] }
          ]
        }
    
    # ruizido_top_5に格納する辞書型を作る
#     mydict = { 
#           'id': qa_dev[idx]['title'],
#           'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
#           'search' : [
#                         { 'top1_id':  get_qa_train[0]['_source']['id'], 'top1_ans':  get_qa_train[0]['_source']['question'] },
#                         { 'top2_id':  get_qa_train[1]['_source']['id'], 'top2_ans':  get_qa_train[1]['_source']['question'] },
#                         { 'top3_id':  get_qa_train[2]['_source']['id'], 'top3_ans':  get_qa_train[2]['_source']['question'] },
#                         { 'top4_id':  get_qa_train[3]['_source']['id'], 'top4_ans':  get_qa_train[3]['_source']['question'] },
#                         { 'top5_id':  get_qa_train[4]['_source']['id'], 'top5_ans':  get_qa_train[4]['_source']['question'] }
#           ]
#           }
    tutoring_search_top_5.append(mydict)
    
    if qa_dev[idx]['title'] == get_id_and_ans[0][0]:
        true_count += 1
    else:
        false_count += 1


print("------結果-------")
print(true_count)
print(false_count)
print("精度：", true_count / (true_count + false_count))

------結果-------
66
34
精度： 0.66


In [62]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data


pickle_dump(tutoring_search_top_5, './tutoring_search2_top_5.pickle')
print(pickle_load('./tutoring_search2_top_5.pickle'))

[{'id': '0610', 'q': 'バイアスとはなんですか', 'search': [{'top1_id': '0610', 'top1_ans': 'バイアスってなんですか'}, {'top2_id': '0610', 'top2_ans': 'バイアスと分散はどのような関係ですか'}, {'top3_id': '0305', 'top3_ans': 'バイアスって何ですか'}, {'top4_id': '0311', 'top4_ans': 'エントロピーとはなんですか'}, {'top5_id': '0114', 'top5_ans': 'クラスタリングとはなんですか'}]}, {'id': '0306', 'q': '候補削除アルゴリズムとはなんですか', 'search': [{'top1_id': '0306', 'top1_ans': '候補削除アルゴリズムってなんですか'}, {'top2_id': '0204', 'top2_ans': '次元削減とはなんですか'}, {'top3_id': '0305', 'top3_ans': 'FIND-Sアルゴリズムとはなんですか'}, {'top4_id': '0204', 'top4_ans': 'なぜ次元を削減するんですか'}, {'top5_id': '0204', 'top5_ans': 'なぜ前処理で次元削減を行うのですか'}]}, {'id': '1502', 'q': '強化学習とは何ですか', 'search': [{'top1_id': '1502', 'top1_ans': '強化学習とはなんですか'}, {'top2_id': '1501', 'top2_ans': '強化学習はなぜ中間的学習なのですか'}, {'top3_id': '1502', 'top3_ans': '強化学習は例えば何に使われていますか'}, {'top4_id': '1501', 'top4_ans': '強化学習が中間的学習という位置づけにある理由はなんですか'}, {'top5_id': '1001', 'top5_ans': 'アンサンブル学習とは何ですか'}]}, {'id': '0605', 'q': '決定係数の定義は何ですか', 'search': [{'top1_id': '0605

In [84]:
################################################################################################
#
#                       パッセージ直接検索
#
################################################################################################

In [101]:
es.indices.delete(index="context")

{'acknowledged': True}

In [102]:
import numpy as np
import pandas as pd
df_context = pd.read_csv('context.csv',dtype=str)
# anyでNaNが一つでもある行を削除する
df_context = df_context.dropna(how='any')
# id が飛び番号になるので振り直しする
df_context = df_context.reset_index()


df_context_copy = df_context.copy()
df_context_copy

import re

# %表記を消して、改行も消して文字を詰める

for i, col in enumerate(df_context['context']):
#     print(i)
#     print("------------------------------------------------------------------")
#     print(col)
    col_tmp = re.sub(r'\\%', "-TMP-", col) 
    col_tmp = re.sub(r'%.*\n', "", col_tmp) 
    # col_tmp = re.sub(r' ', "", col_tmp) 
    col_tmp = re.sub(r'-TMP-', "\%", col_tmp) 
#     print("---------------------------変換後-----------------------------------")
    col_tmp = re.sub(r'\n', "", col_tmp)
#     print(col_tmp)
    df_context_copy['context'][i] = col_tmp
    
    
for i, item in df_context_copy.iterrows():
    print(i, item)
    es.index(index="context", 
             doc_type="context", 
             body={ 
                 "context": item['context'],
                 "id": item['id_title']
             })
    
    


C:\Users\b6122\Anaconda3\envs\ginza\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0 index                                                       0
id_title                                                 0101
context     我々人間は，日々五感を通して得られる情報から，対象を分類する能力や，事象の背後にある規則性を...
Name: 0, dtype: object
1 index                                                       1
id_title                                                 0102
context     近年，日常生活やビジネスにおけるさまざまな場面で人工知能 (aritificial inte...
Name: 1, dtype: object
2 index                                                       2
id_title                                                 0103
context     機械学習の出番は，簡単には規則化できない複雑なデータが大量にあり，そこから得られる知見が有用...
Name: 2, dtype: object
3 index                                                       3
id_title                                                 0104
context     データから規則や知見を得る機械学習技術のなかでも，特に深層学習 (deep learning...
Name: 3, dtype: object
4 index                                                       4
id_title                                                 0105
context     ここでは，もう少し詳細に機械学習の中

79 index                                                      85
id_title                                                 0607
context     また，パラメータ$\bm{w}$の絶対値を正則化項とするものをLasso回帰とよびます．一般...
Name: 79, dtype: object
80 index                                                      86
id_title                                                 0608
context     Ridge回帰とLasso回帰における正則化の振る舞いの違いを図6.4に示します．Ridge...
Name: 80, dtype: object
81 index                                                      87
id_title                                                 0609
context     前節で説明した最小二乗法は，回帰式を高次方程式に置き換えてもそのまま適用できます．一般に，特...
Name: 81, dtype: object
82 index                                                      88
id_title                                                 0610
context     ここで，バイアスと分散の関係を考えてみます．バイアスは真のモデルとの距離，分散は学習結果の散...
Name: 82, dtype: object
83 index                                                      89
id_title                                                 0611
context     回帰木とは，識別に

134 index                                                     153
id_title                                                 1012
context     作成する識別器に対して，誤りを減らすことに特化させるために，個々のデータに対して重みを設定し...
Name: 134, dtype: object
135 index                                                     154
id_title                                                 1013
context     このようなブースティングアルゴリズムの代表的なものにAdaBoostがあります．AdaBoo...
Name: 135, dtype: object
136 index                                                     155
id_title                                                 1014
context     前章で，Adaboostは，特定の損失関数を最小化していることと同じであることを説明しました...
Name: 136, dtype: object
137 index                                                     156
id_title                                                 1015
context     式(10.4)で，差が最小になるものを求めている部分を，損失関数の勾配に置き換えた式(10....
Name: 137, dtype: object
138 index                                                     157
id_title                                                 1101
context     

189 index                                                     214
id_title                                                 1409
context     自己学習の問題点は，自分が出した誤りを指摘してくれる他人がいない，というたとえができます．そ...
Name: 189, dtype: object
190 index                                                     215
id_title                                                 1410
context     共訓練の特徴は，学習初期の誤りに強いということが挙げられます．欠点としては，それぞれが識別器...
Name: 190, dtype: object
191 index                                                     216
id_title                                                 1411
context     ラベル特徴の教師あり／なしの混合データに対する半教師あり学習のように，特徴的なラベルが同じク...
Name: 191, dtype: object
192 index                                                     217
id_title                                                 1412
context     ラベル伝搬法の考え方は，特徴空間上のデータをノードとみなし，類似度に基づいたグラフ構造を構築...
Name: 192, dtype: object
193 index                                                     218
id_title                                                 1413
context     

In [112]:
true_count = 0
false_count = 0


search_top_5 = []  # { 'id':'0101', 
                    #    'q':'多階層ニューラルネットワークとは何か', 
                    #    'ruizido' : [
                    #       { 'top1_id': '0102, 'top1_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #       { 'top2_id': '0102, 'top2_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #          ...
                    #     ]

with open('qa_dev1.json', 'r',encoding="utf-8") as f:
    qa_dev = json.load(f)
    
qa_dev = qa_dev['data']

for idx in range(len(qa_dev)):
    
# sudachiを使うver    
#     body = {
#         "query" : {
#             "bool": {
#                 "must": [
#                     {
#                       "query_string": {
#                         "analyzer": "sudachi_analyzer",
#                         "query": qa_dev[idx]['paragraphs'][0]['qas'][0]['question']
#                       }
#                     }
#                 ]            
#             }
#         },
#         "highlight": {
#             "fields": {
#                 "itemCaption": {}
#             }
#         }
#     }

#  sudachiを使わないver
    body = {
        "query" : {
            "bool": {
                "must": [
                ]            
            }
        },
        "highlight": {
            "fields": {
                "itemCaption": {}
            }
        }
    }

    body['query']['bool']['must'].append(
        { 
            "bool": {
                "should": [
                    { "match": { "context": qa_dev[idx]['paragraphs'][0]['qas'][0]['question'] } }
                ]
            }
        }
    )

    print("------------devの質問---------------")
    print(qa_dev[idx]['title'], qa_dev[idx]['paragraphs'][0]['qas'][0]['question'])
    print("------------trainの検索結果------------")
    result = es.search(index='context', body=body, size=5) # ここをかえる！！！！
    result_num = result['hits']['total']['value']
    get_qa_train = result['hits']['hits']
#     print(get_qa_train[0]['_source']['id'],get_qa_train[0]['_source']['question'])
#     print(get_qa_train[1]['_source']['id'],get_qa_train[1]['_source']['question'])
#     print(get_qa_train[2]['_source']['id'],get_qa_train[2]['_source']['question'])
#     print(get_qa_train)
    
    # ruizido_top_5に格納する辞書型を作る
    mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_qa_train[0]['_source']['id']},
                        { 'top2_id':  get_qa_train[1]['_source']['id']},
                        { 'top3_id':  get_qa_train[2]['_source']['id']},
                        { 'top4_id':  get_qa_train[3]['_source']['id']},
                        { 'top5_id':  get_qa_train[4]['_source']['id']}
          ]
          }
    search_top_5.append(mydict)
    
    if qa_dev[idx]['title'] == get_qa_train[0]['_source']['id']:
        true_count += 1
    else:
        false_count += 1


print("------結果-------")
print(true_count)
print(false_count)
print("精度：", true_count / (true_count + false_count))

------------devの質問---------------
0302 クラスって何ですか
------------trainの検索結果------------
------------devの質問---------------
1302 系列ラベリングでの問題は何がありますか
------------trainの検索結果------------
------------devの質問---------------
0710 二次元から三次元の変換・写像で気をつけることはなんですか
------------trainの検索結果------------
------------devの質問---------------
0906 多階層ニューラルネットワークにおいて、3階層ニューラルネットワークに1層加えて非線形にすることが十分でないのはなぜか
------------trainの検索結果------------
------------devの質問---------------
0707 線形分離可能でない場合はどうすれば良いですか
------------trainの検索結果------------
------------devの質問---------------
1407 自己学習とは何ですか
------------trainの検索結果------------
------------devの質問---------------
0313 過学習とはなんですか
------------trainの検索結果------------
------------devの質問---------------
0901 深層学習とは何か
------------trainの検索結果------------
------------devの質問---------------
0404 事前確率とはなんですか
------------trainの検索結果------------
------------devの質問---------------
0810 勾配消失問題とは何ですか
------------trainの検索結果------------
------------devの質問---------------
0117 半教師あり学習はどんなときに使われますか
---

In [113]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data


pickle_dump(search_top_5, './passage_top_5.pickle')
print(pickle_load('./passage_top_5.pickle'))

[{'id': '0302', 'q': 'クラスって何ですか', 'search': [{'top1_id': '0114'}, {'top2_id': '0406'}, {'top3_id': '0404'}, {'top4_id': '0402'}, {'top5_id': '1310'}]}, {'id': '1302', 'q': '系列ラベリングでの問題は何がありますか', 'search': [{'top1_id': '1302'}, {'top2_id': '1301'}, {'top3_id': '1303'}, {'top4_id': '0916'}, {'top5_id': '1309'}]}, {'id': '0710', 'q': '二次元から三次元の変換・写像で気をつけることはなんですか', 'search': [{'top1_id': '0710'}, {'top2_id': '1403'}, {'top3_id': '0112'}, {'top4_id': '0205'}, {'top5_id': '0908'}]}, {'id': '0906', 'q': '多階層ニューラルネットワークにおいて、3階層ニューラルネットワークに1層加えて非線形にすることが十分でないのはなぜか', 'search': [{'top1_id': '0906'}, {'top2_id': '0802'}, {'top3_id': '0803'}, {'top4_id': '0810'}, {'top5_id': '0903'}]}, {'id': '0707', 'q': '線形分離可能でない場合はどうすれば良いですか', 'search': [{'top1_id': '0710'}, {'top2_id': '0508'}, {'top3_id': '0507'}, {'top4_id': '0707'}, {'top5_id': '0706'}]}, {'id': '1407', 'q': '自己学習とは何ですか', 'search': [{'top1_id': '1407'}, {'top2_id': '1408'}, {'top3_id': '1410'}, {'top4_id': '1409'}, {'top5_id': '0908'}]}, {

In [114]:
################################################################################################
#
#                                     以上フラットな検索
#
#                    以下ではチュータリングシステムにおける検索精度を測る
#
################################################################################################

In [115]:
es = Elasticsearch("localhost:9200")



true_count = 0
false_count = 0


tutoring_search_top_5 = []  # { 'id':'0101', 
                    #    'q':'多階層ニューラルネットワークとは何か', 
                    #    'ruizido' : [
                    #       { 'top1_id': '0102, 'top1_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #       { 'top2_id': '0102, 'top2_ans': '多層パーセプトロンあるいはニューラルネットワーク'},
                    #          ...
                    #     ]

for idx in range(len(qa_dev)):
    
# sudachiを使うver    
#     body = {
#         "query" : {
#             "bool": {
#                 "must": [
#                     {
#                       "query_string": {
#                         "analyzer": "sudachi_analyzer",
#                         "query": qa_dev[idx]['paragraphs'][0]['qas'][0]['question']
#                       }
#                     }
#                 ]            
#             }
#         },
#         "highlight": {
#             "fields": {
#                 "itemCaption": {}
#             }
#         }
#     }

#  sudachiを使わないver
    body = {
        "query" : {
            "bool": {
                "must": [
                ]            
            }
        },
        "highlight": {
            "fields": {
                "itemCaption": {}
            }
        }
    }

    body['query']['bool']['must'].append(
        { 
            "bool": {
                "should": [
                    { "match": { "context": qa_dev[idx]['paragraphs'][0]['qas'][0]['question'] } }
                ]
            }
        }
    )

#     print("------------devの質問---------------")
#     print(qa_dev[idx]['title'], qa_dev[idx]['paragraphs'][0]['qas'][0]['question'])
#     print("------------trainの検索結果------------")
    result = es.search(index='context', body=body, size=50)
    result_num = result['hits']['total']['value']
    get_qa_train = result['hits']['hits']
    
    get_id_and_ans = []
    
    for one_get_qa_train in get_qa_train:
        
        if int(qa_dev[idx]['title']) >= int(one_get_qa_train['_source']['id']):
            
            get_id_and_ans.append([one_get_qa_train['_source']['id']])
            
        if len(get_id_and_ans) == 5:
            break
    

    if len(get_id_and_ans) == 1:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  None },
                        { 'top2_id':  None, 'top2_ans':  None },
                        { 'top3_id':  None, 'top3_ans':  None },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 2:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans': None },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans': None },
                        { 'top3_id':  None, 'top3_ans':  None },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 3:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  None },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  None },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans':  None },
                        { 'top4_id':  None, 'top4_ans':  None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
    
    if len(get_id_and_ans) == 4:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans': None },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans': None },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans': None },
                        { 'top4_id':  get_id_and_ans[3][0], 'top4_ans': None },
                        { 'top5_id':  None, 'top5_ans':  None }
          ]
        }
        
    if len(get_id_and_ans) == 5:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  get_id_and_ans[0][0], 'top1_ans':  None },
                        { 'top2_id':  get_id_and_ans[1][0], 'top2_ans':  None },
                        { 'top3_id':  get_id_and_ans[2][0], 'top3_ans':  None },
                        { 'top4_id':  get_id_and_ans[3][0], 'top4_ans':  None },
                        { 'top5_id':  get_id_and_ans[4][0], 'top5_ans':  None }
          ]
        }
    
    # ruizido_top_5に格納する辞書型を作る
#     mydict = { 
#           'id': qa_dev[idx]['title'],
#           'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
#           'search' : [
#                         { 'top1_id':  get_qa_train[0]['_source']['id'], 'top1_ans':  get_qa_train[0]['_source']['question'] },
#                         { 'top2_id':  get_qa_train[1]['_source']['id'], 'top2_ans':  get_qa_train[1]['_source']['question'] },
#                         { 'top3_id':  get_qa_train[2]['_source']['id'], 'top3_ans':  get_qa_train[2]['_source']['question'] },
#                         { 'top4_id':  get_qa_train[3]['_source']['id'], 'top4_ans':  get_qa_train[3]['_source']['question'] },
#                         { 'top5_id':  get_qa_train[4]['_source']['id'], 'top5_ans':  get_qa_train[4]['_source']['question'] }
#           ]
#           }


    if len(get_id_and_ans) == 0:
        mydict = { 
          'id': qa_dev[idx]['title'],
          'q' : qa_dev[idx]['paragraphs'][0]['qas'][0]['question'],
          'search' : [
                        { 'top1_id':  None },
                        { 'top2_id':  None },
                        { 'top3_id':  None },
                        { 'top4_id':  None },
                        { 'top5_id':  None }
          ]
        }
        
    tutoring_search_top_5.append(mydict)
    
    if len(get_id_and_ans) == 0:
        false_count += 1
        print(qa_dev[idx]['paragraphs'][0]['qas'][0]['question'])
        print( int(qa_dev[idx]['title']) >= int(one_get_qa_train['_source']['id']))
        continue
    
    if qa_dev[idx]['title'] == get_id_and_ans[0][0]:
        true_count += 1
    else:
        false_count += 1


print("------結果-------")
print(true_count)
print(false_count)
print("精度：", true_count / (true_count + false_count))

------結果-------
80
20
精度： 0.8


In [116]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data


pickle_dump(tutoring_search_top_5, './tutoring_passage_top_5.pickle')
print(pickle_load('./tutoring_passage_top_5.pickle'))

[{'id': '0302', 'q': 'クラスって何ですか', 'search': [{'top1_id': '0114', 'top1_ans': None}, {'top2_id': '0117', 'top2_ans': None}, {'top3_id': '0111', 'top3_ans': None}, {'top4_id': '0211', 'top4_ans': None}, {'top5_id': '0101', 'top5_ans': None}]}, {'id': '1302', 'q': '系列ラベリングでの問題は何がありますか', 'search': [{'top1_id': '1302', 'top1_ans': None}, {'top2_id': '1301', 'top2_ans': None}, {'top3_id': '0916', 'top3_ans': None}, {'top4_id': '0906', 'top4_ans': None}, {'top5_id': '0604', 'top5_ans': None}]}, {'id': '0710', 'q': '二次元から三次元の変換・写像で気をつけることはなんですか', 'search': [{'top1_id': '0710', 'top1_ans': None}, {'top2_id': '0112', 'top2_ans': None}, {'top3_id': '0205', 'top3_ans': None}, {'top4_id': '0502', 'top4_ans': None}, {'top5_id': '0213', 'top5_ans': None}]}, {'id': '0906', 'q': '多階層ニューラルネットワークにおいて、3階層ニューラルネットワークに1層加えて非線形にすることが十分でないのはなぜか', 'search': [{'top1_id': '0906', 'top1_ans': None}, {'top2_id': '0802', 'top2_ans': None}, {'top3_id': '0803', 'top3_ans': None}, {'top4_id': '0810', 'top4_ans': None}